### 1. Verify that all data reported by Jin is in

In [1]:
import scIB

In [2]:
import os
import scanpy as sc
from os.path import join
from os import listdir
import anndata
import numpy as np
import scipy
import os

In [3]:
# convert counts into float32
# Convenience method for computing the size of objects
def print_size_in_MB(x):
    print('{:.3} MB'.format(x.__sizeof__()/1e6))

In [52]:
# datadir = '/storage/groups/ml01/datasets/projects/20210318_retinal_data_integration_ignacio.ibarra_malte.luecken'
datadir = '/mnt/f/workspace/theislab/retina/data/RNA'
# outdir = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/scran'
outdir = '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran'
os.path.exists(outdir), os.path.exists(datadir)

(True, True)

In [53]:
filenames = [f for f in os.listdir(datadir)]

In [54]:
# filenames_md5 = [f.strip() for f in open(os.path.join(datadir, 'md5sum.txt'))]
filenames_md5 = [f.strip() for f in os.listdir(datadir) if f.endswith(".h5ad")]
filenames_md5

# files = set()
# for qi in filenames_md5:
#     md5, fi = qi.split('  ')
#     found = os.path.exists(os.path.join(datadir, fi))
#     if not found:
#         print('not found', fi)
#     files.add(fi)

['Chang_GSM5866081.h5ad',
 'Chang_GSM5866082.h5ad',
 'Chang_GSM5866083.h5ad',
 'Chang_GSM5866084.h5ad',
 'Chang_GSM5866085.h5ad',
 'Chang_GSM5866086.h5ad',
 'Chang_GSM5866087.h5ad',
 'Chang_GSM5866088.h5ad',
 'Chen_a_10x3_Lobe_19_D003_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D003_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D005_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D005_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D006_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D006_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D007_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D007_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D008_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D008_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D009_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D009_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D010_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D010_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D011_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D011_Nu.h5ad',
 'Chen_a_10x3_Lobe_19_D019_NeuN.h5ad',
 'Chen_a_10x3_Lobe_19_D019_Nu.h5ad',
 'Chen_a_10x3_Lobe_D005_13_NeuN.h5ad',
 'Chen_a_10x3_Lobe_D005_13_Nu.h5ad',
 'Chen_a_10x3_Lobe_D009_13_NeuN.

**The following files are listed but for some reason not found anymore. Consider deleting (Request to Jin first)**

In [55]:
filenames_by_dataset = {}
for f in filenames_md5:
    # dataset, filename = f.split(' ')[-1].split('/')[-2:]
    dataset = f.split('_')[0] if not "Chen" in f else f.split('_')[0] + '_' + f.split('_')[1]
    filename = f
    print(dataset, filename)
    if not dataset in filenames_by_dataset:
        filenames_by_dataset[dataset] = []
    filenames_by_dataset[dataset].append(filename)

Chang Chang_GSM5866081.h5ad
Chang Chang_GSM5866082.h5ad
Chang Chang_GSM5866083.h5ad
Chang Chang_GSM5866084.h5ad
Chang Chang_GSM5866085.h5ad
Chang Chang_GSM5866086.h5ad
Chang Chang_GSM5866087.h5ad
Chang Chang_GSM5866088.h5ad
Chen_a Chen_a_10x3_Lobe_19_D003_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D003_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D005_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D005_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D006_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D006_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D007_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D007_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D008_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D008_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D009_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D009_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D010_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D010_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D011_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D011_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_19_D019_NeuN.h5ad
Chen_a Chen_a_10x3_Lobe_19_D019_Nu.h5ad
Chen_a Chen_a_10x3_Lobe_D005_13_NeuN.h

### scran normalization

In [56]:
path_preprocessing = '/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py'
os.path.exists(path_preprocessing)

True

In [64]:
def execute_preprocessing(input_path, output_path):
    print('')
    path_preprocessing = '/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py'
    cmd = 'python %s -i %s -o %s' % (path_preprocessing, input_path, output_path)
    
    try:
        print(cmd)
        os.system(cmd)
    except Exception as err:
        print('something went wrong...')
        print(err)

In [58]:
import multiprocessing
from multiprocessing import Process
from multiprocessing import Manager

def run(function, input_list, n_cores, log_each=None, log=False):
    print(('run function %s with n_cores = %i' % (function, n_cores)))
    print(function)
    # print 'with input list of len'
    # print len(input_list)
    # print 'in groups of %d threads' % n_threads

    assert n_cores <= 20

    # the type of input_list has to be a list. If not
    # then it can a single element list and we cast it to list.
    if not isinstance(type(input_list[0]), type(list)):
        input_list = [[i] for i in input_list]

    n_groups = int(len(input_list) / n_cores + 1)
    # print 'n groups', n_groups

    n_done = 0
    for group_i in range(n_groups):
        start, end = group_i * n_cores, (group_i + 1) * n_cores
        # print 'start', start, 'end', end

        threads = [None] * (end - start)
        for i, pi in enumerate(range(start, min(end, len(input_list)))):
            next_args = input_list[pi]
            if log:
                print(next_args)
            # print next_kmer
            threads[i] = Process(target=function, args=next_args)
            # print 'starting process #', i
            threads[i].start()

        # print  threads
        # print 'joining threads...'
        # do some other stuff
        for i in range(len(threads)):
            if threads[i] is None:
                continue
            threads[i].join()

            n_done += 1
            if log_each is not None and log_each % n_done == 0:
                print('Done %i so far' % n_done)
    print('done...')


In [59]:
from os.path import join
arguments = []

for dataset in filenames_by_dataset:
    # print(dataset)
    for filename in filenames_by_dataset[dataset]:
        input_file = join(datadir, filename)
        next_outdir = join(outdir, dataset)
        # print(next_outdir)
        
        if not os.path.exists(next_outdir):
            os.mkdir(next_outdir)
            
        output_file = join(next_outdir, filename)

        # print(os.path.exists(output_file), output_file)
        if os.path.exists(output_file):
            # print(os.path.exists(output_file), 'skip...')
            continue
        
        # print(input_file)
        # print(output_file)
        # print('')
        
        arguments.append([input_file, output_file])
        # ad = sc.read_h5ad(join(datadir, p))
    

In [60]:
arguments

[['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866081.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang_GSM5866081.h5ad'],
 ['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866082.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang_GSM5866082.h5ad'],
 ['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866083.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang_GSM5866083.h5ad'],
 ['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866084.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang_GSM5866084.h5ad'],
 ['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866085.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang_GSM5866085.h5ad'],
 ['/mnt/f/workspace/theislab/retina/data/RNA/Chang_GSM5866086.h5ad',
  '/mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Chang/Chang

In [61]:
# this is a test. maintain commented after finishing
# !python ../../scib/scripts/preprocessing_remove_empty.py -i /storage/groups/ml01/datasets/projects/20210318_retinal_data_integration_ignacio.ibarra_malte.luecken/Wong/Retina_2B.h5ad -o /mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/scran/Wong/Retina_2B.h5ad

In [62]:
print(len(arguments))
arguments = sorted(arguments, key=lambda x: os.path.getsize(x[0]))

152


In [73]:
h5ad_path = arguments[0][0]
import scanpy as sc
adata = sc.read_h5ad(h5ad_path)

In [80]:
adata.shape

(129, 36601)

In [108]:
run(execute_preprocessing, arguments, n_cores=5)

run function <function execute_preprocessing at 0x7f9899b2d440> with n_cores = 5
<function execute_preprocessing at 0x7f9899b2d440>

python /mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py -i /mnt/f/workspace/theislab/retina/data/RNA/Sanes_H3FoveaS3.h5ad -o /mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Sanes/Sanes_H3FoveaS3.h5ad
python /mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py -i /mnt/f/workspace/theislab/retina/data/RNA/Hafler_GSM4081524.h5ad -o /mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Hafler/Hafler_GSM4081524.h5ad


python /mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py -i /mnt/f/workspace/theislab/retina/data/RNA/Scheetz_GSM3745992.h5ad -o /mnt/f/workspace/theislab/retina/data/integration_march_2021/scran/Scheetz/Scheetz_GSM3745992.h5ad


python /mnt/c/Users/ignacio.ibarra/Dropbox/

Traceback (most recent call last):
  File "/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scripts/preprocessing.py", line 27, in <module>
    pp.normalize(adata)
  File "/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/HECA-scib-pipeline/scIB/preprocessing.py", line 190, in normalize
    sc.pp.neighbors(adata_pp)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/scanpy/neighbors/__init__.py", line 147, in neighbors
    random_state=random_state,
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/scanpy/neighbors/__init__.py", line 792, in compute_neighbors
    X, n_neighbors, random_state, metric=metric, metric_kwds=metric_kwds
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/scanpy/neighbors/__init__.py", line 312, in compute_neighbors_umap
    verbose=verbose,
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/umap/umap_.py", line 340, in nearest_neighbors
 

Normalisation
Normalisation


 154, in propagate
    constraint(typeinfer)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/numba/core/typeinfer.py", line 566, in __call__
    self.resolve(typeinfer, typevars, fnty)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/numba/core/typeinfer.py", line 586, in resolve
    sig = typeinfer.resolve_call(fnty, pos_args, kw_args)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/numba/core/typeinfer.py", line 1538, in resolve_call
    return self.context.resolve_function_type(fnty, pos_args, kw_args)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/numba/core/typing/context.py", line 194, in resolve_function_type
    res = self._resolve_user_function_type(func, args, kws)
  File "/home/rio/miniconda3/envs/scIB-python/lib/python3.7/site-packages/numba/core/typing/context.py", line 246, in _resolve_user_function_type
    return func.get_call_type(self, args, kws)
  File "/home/rio/m

KeyboardInterrupt: 